# B-Trees

A specialized m-way tree designed to optimize data access.

### Properties:
- All leaf nodes of a B-tree are at the same level, i.e. they have the same depth (height of the tree).
- The keys of each node of a B-tree (in case of multiple keys), should be stored in the ascending order.
- In a B-tree, all non-leaf nodes (except root node) should have at least $\frac{m}{2}$ children.
- All nodes (except root node) should have at least $\frac{m}{2} - 1$ keys.
- If the root node is a leaf node (only node in the tree), then it will have no children and will have at least one key. If the root node is a non-leaf node, then it will have at least '2 children' and at least one key.
- A non-leaf node with n-1 key values should have n non NULL children.

Example of a B-Tree of order 5:


                              [20]
                           /        \
                     [10]          [30, 40]
                    /  |  \        /   |   \
                 [5]  [15] [25]  [35] [45] [50]


# Insertion Operation
To insert a new key, we go down from root to leaf. Before traversing down to a node, we first check if the node is full. If the node is full, we split it to create space. Following is the complete algorithm.

### B-Tree Insert Algorithm

```plaintext
procedure B-Tree-Insert(Node x, Key k)
    find i such that x.keys[i] > k or i >= numkeys(x)
    
    if x is a leaf then
        Insert k into x.keys at i
    else
        if x.child[i] is full then
            Split x.child[i]
            if k > x.keys[i] then
                i = i + 1
            end if
        end if
        B-Tree-Insert(x.child[i], k)
    end if
end procedure


In [5]:
class BTreeNode:
    def __init__(self, t, leaf):
        self.keys = [None] * (2 * t - 1) # An array of keys
        self.t = t # Minimum degree (defines the range for number of keys)
        self.C = [None] * (2 * t) # An array of child pointers
        self.n = 0 # Current number of keys
        self.leaf = leaf # Is true when node is leaf. Otherwise false

    # A utility function to insert a new key in the subtree rooted with
    # this node. The assumption is, the node must be non-full when this
    # function is called
    def insertNonFull(self, k):
        i = self.n - 1
        if self.leaf:
            while i >= 0 and self.keys[i] > k:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
            self.n += 1
            print(f"Inserted {k} in leaf node: {self.keys[:self.n]}")
        else:
            while i >= 0 and self.keys[i] > k:
                i -= 1
            if self.C[i + 1].n == 2 * self.t - 1:
                self.splitChild(i + 1, self.C[i + 1])
                if self.keys[i + 1] < k:
                    i += 1
            self.C[i + 1].insertNonFull(k)

    # A utility function to split the child y of this node. i is index of y in
    # child array C[].  The Child y must be full when this function is called
    def splitChild(self, i, y):
        z = BTreeNode(y.t, y.leaf)
        z.n = self.t - 1
        for j in range(self.t - 1):
            z.keys[j] = y.keys[j + self.t]
        if not y.leaf:
            for j in range(self.t):
                z.C[j] = y.C[j + self.t]
        y.n = self.t - 1
        for j in range(self.n, i, -1):
            self.C[j + 1] = self.C[j]
        self.C[i + 1] = z
        for j in range(self.n - 1, i - 1, -1):
            self.keys[j + 1] = self.keys[j]
        self.keys[i] = y.keys[self.t - 1]
        self.n += 1
        print(f"Split child node and promoted key {y.keys[self.t - 1]}: {self.keys[:self.n]}")

    # A function to traverse all nodes in a subtree rooted with this node
    def traverse(self):
        for i in range(self.n):
            if not self.leaf:
                self.C[i].traverse()
            print(self.keys[i], end=' ')
        if not self.leaf:
            self.C[i + 1].traverse()

    # A function to search a key in the subtree rooted with this node.
    def search(self, k):
        i = 0
        while i < self.n and k > self.keys[i]:
            i += 1
        if i < self.n and k == self.keys[i]:
            return self
        if self.leaf:
            return None
        return self.C[i].search(k)

# A BTree
class BTree:
    def __init__(self, t):
        self.root = None # Pointer to root node
        self.t = t  # Minimum degree

    # function to traverse the tree
    def traverse(self):
        if self.root != None:
            self.root.traverse()

    # function to search a key in this tree
    def search(self, k):
        return None if self.root == None else self.root.search(k)

    # The main function that inserts a new key in this B-Tree
    def insert(self, k):
        if self.root == None:
            self.root = BTreeNode(self.t, True)
            self.root.keys[0] = k # Insert key
            self.root.n = 1
            print(f"Inserted {k} in root node: {self.root.keys[:self.root.n]}")
        else:
            if self.root.n == 2 * self.t - 1:
                s = BTreeNode(self.t, False)
                s.C[0] = self.root
                s.splitChild(0, self.root)
                i = 0
                if s.keys[0] < k:
                    i += 1
                s.C[i].insertNonFull(k)
                self.root = s
            else:
                self.root.insertNonFull(k)

# Driver program to test above functions
if __name__ == '__main__':
    t = BTree(3) # A B-Tree with minimum degree 3
    keys = [10, 20, 5, 6, 12, 30, 7, 17]
    
    for key in keys:
        print(f"Inserting {key}...")
        t.insert(key)
        print(f"Current tree after inserting {key}: ", end=" ")
        t.traverse()
        print("\n---")

    # Test search
    k = 6
    if t.search(k) != None:
        print(f"{k} is Present")
    else:
        print(f"{k} is Not Present")

    k = 15
    if t.search(k) != None:
        print(f"{k} is Present")
    else:
        print(f"{k} is Not Present")


Inserting 10...
Inserted 10 in root node: [10]
Current tree after inserting 10:  10 
---
Inserting 20...
Inserted 20 in leaf node: [10, 20]
Current tree after inserting 20:  10 20 
---
Inserting 5...
Inserted 5 in leaf node: [5, 10, 20]
Current tree after inserting 5:  5 10 20 
---
Inserting 6...
Inserted 6 in leaf node: [5, 6, 10, 20]
Current tree after inserting 6:  5 6 10 20 
---
Inserting 12...
Inserted 12 in leaf node: [5, 6, 10, 12, 20]
Current tree after inserting 12:  5 6 10 12 20 
---
Inserting 30...
Split child node and promoted key 10: [10]
Inserted 30 in leaf node: [12, 20, 30]
Current tree after inserting 30:  5 6 10 12 20 30 
---
Inserting 7...
Inserted 7 in leaf node: [5, 6, 7]
Current tree after inserting 7:  5 6 7 10 12 20 30 
---
Inserting 17...
Inserted 17 in leaf node: [12, 17, 20, 30]
Current tree after inserting 17:  5 6 7 10 12 17 20 30 
---
6 is Present
15 is Not Present
